In [1]:
import pandas as pd
import datetime

In [2]:
date_str = datetime.datetime.now().strftime("%Y-%m-%d")

print("Date: " + date_str)

Date: 2018-04-11


In [3]:
# Read the Calendar extract to get additional details
cal_1 = pd.read_json('/Users/taj/GitHub/scraping/stayz/WebData/nsw_calendar/stayz_calendar_' + date_str + '.json')

cal_1.set_index('property_id',inplace=True)
cal_1.head()


,calendar,ext_at,features,photos,review_count,review_value
property_id,,,,,,
9082944,"<thead><tr><th scope=""col""><span title=""Monday...",2018-04-11 07:02:49,Books\nChild's high chair\nChildren welcome\nC...,24,174,5.0
9030183,"<thead><tr><th scope=""col""><span title=""Monday...",2018-04-11 07:02:51,Air conditioning\nBalcony\nBooks\nChildren wel...,24,93,4.8
9026665,"<thead><tr><th scope=""col""><span title=""Monday...",2018-04-11 07:02:53,Air conditioning\nBalcony\nChild's high chair\...,16,15,4.9
9152385,"<thead><tr><th scope=""col""><span title=""Monday...",2018-04-11 07:02:54,Air conditioning\nChild's high chair\nChildren...,24,11,5.0
9119781,"<thead><tr><th scope=""col""><span title=""Monday...",2018-04-11 07:02:57,Balcony\nBooks\nChild's high chair\nChildren w...,9,22,-1.0


In [4]:
position_dict = {}
position_index = 0


data_dict = {}


# Step one - Create the dictionary of all possible feature items
for index, row in cal_1.iterrows():
    
    a = row['features']
    
    #print("Features: " + a)
    
    b = a.split('\n')
    
    for b1 in b:
        # Check if it exists in the dictionary
        
        if ( len(b1) > 0 ):
            if b1 in position_dict:
                b1_pos = position_dict[b1]

            else:
                position_dict[b1] = position_index

                b1_pos = position_index

                position_index += 1
            
print(position_dict)



{'Books': 0, "Child's high chair": 1, 'Children welcome': 2, 'Clothes dryer': 3, 'Coffee maker': 4, 'Deck/Patio': 5, 'Dining': 6, 'Dishes and utensils': 7, 'Dishwasher': 8, 'DVD player': 9, 'Fireplace': 10, 'Games': 11, 'Hair dryer': 12, 'Heating': 13, 'Internet access': 14, 'Iron and board': 15, 'Kitchen': 16, 'Lawn/Garden': 17, 'Linens provided': 18, 'Living room': 19, 'Microwave': 20, 'Non-Smoking': 21, 'Outdoor grill': 22, 'Oven': 23, 'Parking': 24, 'Pets allowed': 25, 'Refrigerator': 26, 'Satellite/Cable': 27, 'Stereo': 28, 'Stove': 29, 'Television': 30, 'Tennis': 31, 'Toaster': 32, 'Towels provided': 33, 'Toys': 34, 'Video library': 35, 'Washing machine': 36, 'Wheelchair accessible': 37, 'Air conditioning': 38, 'Balcony': 39, 'Hot tub': 40, 'Video Games': 41, 'Communal pool': 42, 'Heated pool': 43, 'Private pool': 44, 'Wood stove': 45, 'Games room': 46, 'Long-term renters welcome': 47, 'Garage': 48, 'Telephone': 49, 'Sauna': 50, 'Events allowed': 51, 'Fitness room/Equipment': 52,

In [5]:

# Step 2 - scan and create the data
for index, row in cal_1.iterrows():
    #print(index)
    a = row['features']
    b = a.split('\n')
    
    save_features = [0] * len(position_dict)
    
    for b1 in b:    
        if ( len(b1) > 0 ):
            if b1 in position_dict:
                b1_pos = position_dict[b1]
                save_features[b1_pos] = 1
            else:
                print("ERROR: found a new feature not in dictionary: '" + b1 + "'")
    
    if ( len(index) > 0 ):
        data_dict[index] = save_features
    
df1 = pd.DataFrame.from_dict(data_dict)

df2 = df1.T

col_n_1 = position_dict.keys()

col_n_2 = []


for c in col_n_1:
    # Cleaning text
    for char in '\'/-.,\n ':
        c=c.replace(char,'_')

    # All words lower case
    c = c.lower()
    
    col_n_2.append('f_' + c)

df2.columns = col_n_2

df2.head()

,f_books,f_child_s_high_chair,f_children_welcome,f_clothes_dryer,f_coffee_maker,f_deck_patio,f_dining,f_dishes_and_utensils,f_dishwasher,f_dvd_player,...,f_sauna,f_events_allowed,f_fitness_room_equipment,f_indoor_pool,f_ceiling_fan,f_cot,f_eco_friendly,f_pantry_items,f_free_wifi,f_free_fixed_line_internet
10000,1,0,0,0,1,1,1,1,1,1,...,0,0,0,0,1,0,0,0,0,0
100012,0,0,1,1,0,0,1,1,1,0,...,0,0,0,0,1,0,0,0,0,0
100052,1,0,1,1,1,1,1,1,1,1,...,0,0,0,0,1,0,0,1,0,0
100171,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
100172,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Write out the file to CSV so it can be joined in the main branch

csv_out = df2.to_csv()

fh = open('/Users/taj/GitHub/scraping/stayz/WebData/nsw_features/stayz_nsw_features_' + date_str + '.csv', 'w')

fh.write(csv_out)

fh.close()